In [2]:
import numpy as np
import pandas as pd
import pickle 
import sklearn 
import matplotlib.pyplot as plt 

In [3]:
with open('../data/train_data.pickle', 'rb') as f:
    train_data = pickle.load(f)

In [4]:
with open('../data/test_data.pickle', 'rb') as f:
    test_data = pickle.load(f)

In [5]:
with open('../data/scaler.pickle', 'rb') as f:
    scaler = pickle.load(f)

In [6]:
#def evaluate_forecasts(actual, predicted):
train_data.head()

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
date,,,,,,,,
2010-01-02 00:00:00,0.129779,0.352941,0.245902,0.527273,0.333333,0.002290,0.000000,0.0
2010-01-02 01:00:00,0.148893,0.367647,0.245902,0.527273,0.333333,0.003811,0.000000,0.0
2010-01-02 02:00:00,0.159960,0.426471,0.229508,0.545455,0.333333,0.005332,0.000000,0.0
2010-01-02 03:00:00,0.182093,0.485294,0.229508,0.563636,0.333333,0.008391,0.037037,0.0
2010-01-02 04:00:00,0.138833,0.485294,0.229508,0.563636,0.333333,0.009912,0.074074,0.0


In [7]:
test_data.head(-5)

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
date,,,,,,,,
2014-12-18 00:00:00,0.181087,0.397059,0.213115,0.709091,0.000000,0.002290,0.0,0.0
2014-12-18 01:00:00,0.171026,0.397059,0.196721,0.709091,0.666667,0.000752,0.0,0.0
2014-12-18 02:00:00,0.160966,0.397059,0.196721,0.709091,0.666667,0.003811,0.0,0.0
2014-12-18 03:00:00,0.146881,0.382353,0.163934,0.727273,0.666667,0.006870,0.0,0.0
2014-12-18 04:00:00,0.125755,0.382353,0.180328,0.709091,0.666667,0.012219,0.0,0.0
...,...,...,...,...,...,...,...,...
2014-12-31 14:00:00,0.009054,0.191176,0.327869,0.745455,0.666667,0.334547,0.0,0.0
2014-12-31 15:00:00,0.011066,0.205882,0.327869,0.745455,0.666667,0.349825,0.0,0.0
2014-12-31 16:00:00,0.008048,0.250000,0.311475,0.745455,0.666667,0.365103,0.0,0.0


In [8]:
def generate_sequence(df,N, window_size):
    '''Model expects the target labels to have two 
        dimensions with shape (batch_size,output_size). 
        - batch_size is the number of samples in each sequence
        - output_size is the number of target values per sample'''
        
    #We generate sequences of size 24
    X_sequences = [df.iloc[i:i+window_size].values for i in range(N - window_size)]
    #And for each sequence evaluate agains the pollution value following each sequence
    Y_values = [df.iloc[i+window_size]['pollution'] for i in range(N - window_size)]


    return np.array(X_sequences).astype(np.float32), np.array(Y_values).astype(np.float32).reshape(-1,1)



In [9]:
window_size = 12
N= len(train_data)
X_train, y_train = generate_sequence(train_data,N, window_size)
print(X_train.shape, y_train.shape)

M=len(test_data)
X_test, y_test = generate_sequence(test_data,M,window_size)
print(X_test.shape,y_test.shape)




(43452, 12, 8) (43452, 1)
(324, 12, 8) (324, 1)


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint



/Users/matt/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Standard LSTM

In [ ]:
n_steps = X_train.shape[1]
n_features = X_train.shape[2]

MV_LSTM = Sequential()
MV_LSTM.add(Input(shape =(n_steps, n_features)))
MV_LSTM.add(LSTM(50))
#MV_LSTM.add(Droput(0.2)) prevents overfitting by randomly dropping out 20% of neurons 

MV_LSTM.add(Dense(1))

#Compile the model
metrics="RootMeanSquaredError()"
optimzer="adam"
loss = 'mse'

MV_LSTM.compile(optimizer=Adam(learning_rate = 0.001), loss=loss, metrics = [RootMeanSquaredError()])


MV_LSTM.summary()

In [ ]:
# fit model
history = MV_LSTM.fit(X_train, y_train, epochs=150, verbose=1)

## Relu layer

Without BatchNormaliztion

In [ ]:
n_steps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = y_train.shape[1]
MV_LSTM = Sequential()
MV_LSTM.add(Input(shape =(n_steps, n_features)))
MV_LSTM.add(LSTM(32,return_sequences=True))
MV_LSTM.add(Dropout(0.2)) #Prevent overfitting
MV_LSTM.add(LSTM(16, return_sequences=False))
#MV_LSTM.add(BatchNormalization()) #Normalize outputs
MV_LSTM.add(Dense(5, activation='relu')) # Small intermediate Dense layer
MV_LSTM.add(Dense(n_outputs)) #Dense output layer with 1 unit (regression problem)

#Compile the model
metrics="RootMeanSquaredError()"
optimzer="adam"
loss = 'mse'

MV_LSTM.compile(optimizer=Adam(learning_rate = 0.001), loss=loss, metrics = [RootMeanSquaredError()])


MV_LSTM.summary()


2024-11-13 12:59:43.652977: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-11-13 12:59:43.653032: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-11-13 12:59:43.653045: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-11-13 12:59:43.653452: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-13 12:59:43.653477: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 24, 32)         │         5,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 16)             │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │            85 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,347 (32.61 KB)

 Trainable params: 8,347 (32.61 KB)

 Non-trainable params: 0 (0.00 B)

With BatchNormalization- No effect

In [ ]:
n_steps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = y_train.shape[1]
MV_LSTM = Sequential()
MV_LSTM.add(Input(shape =(n_steps, n_features)))
MV_LSTM.add(LSTM(32,return_sequences=True))
MV_LSTM.add(Dropout(0.2)) #Prevent overfitting
MV_LSTM.add(LSTM(16, return_sequences=False))
MV_LSTM.add(BatchNormalization()) #Normalize outputs
MV_LSTM.add(Dense(5, activation='relu')) # Small intermediate Dense layer
MV_LSTM.add(Dense(n_outputs)) #Dense output layer with 1 unit (regression problem)

#Compile the model
metrics="RootMeanSquaredError()"
optimzer="adam"
loss = 'mse'

MV_LSTM.compile(optimizer=Adam(learning_rate = 0.001), loss=loss, metrics = [RootMeanSquaredError()])


MV_LSTM.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 24, 32)         │         5,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 24, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 16)             │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │            85 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,411 (32.86 KB)

 Trainable params: 8,379 (32.73 KB)

 Non-trainable params: 32 (128.00 B)

## Stacked With BatchNormalization

In [ ]:
n_steps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = y_train.shape[1]
MV_LSTM = Sequential()
MV_LSTM.add(Input(shape =(n_steps, n_features)))
MV_LSTM.add(LSTM(32,return_sequences=True))
MV_LSTM.add(Dropout(0.2)) #Prevent overfitting
MV_LSTM.add(LSTM(16, return_sequences=False))
MV_LSTM.add(BatchNormalization()) #Normalize outputs
#MV_LSTM.add(Dense(5, activation='relu')) # Small intermediate Dense layer
MV_LSTM.add(Dense(n_outputs)) #Dense output layer with 1 unit (regression problem)

#Compile the model
metrics="RootMeanSquaredError()"
optimzer="adam"
loss = 'mse'

MV_LSTM.compile(optimizer=Adam(learning_rate = 0.001), loss=loss, metrics = [RootMeanSquaredError()])


MV_LSTM.summary()

In [ ]:
test4 = MV_LSTM.fit(X_train, y_train, epochs=150, validation_split=0.1, batch_size=32,shuffle=False)

In [ ]:
test_predictions = MV_LSTM.predict(X_test).flatten()
actual_predictions = y_test.flatten()
test_predictions.shape, actual_predictions.shape

test_results = pd.DataFrame(data={
        'Model Predictions': test_predictions,
        'Actual':actual_predictions})
test_results.head()

## Early Stopping

In [12]:
n_steps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = y_train.shape[1]

MV_LSTM = Sequential()
MV_LSTM.add(Input(shape =(n_steps, n_features)))
MV_LSTM.add(LSTM(32,return_sequences=True))
MV_LSTM.add(Dropout(0.2)) #Prevent overfitting
MV_LSTM.add(LSTM(16, return_sequences=False))

MV_LSTM.add(Dense(n_outputs)) #Dense output layer with 1 unit (regression problem)

#Compile the model
metrics="RootMeanSquaredError()"
optimzer="adam"
loss = 'mse'

MV_LSTM.compile(optimizer=Adam(learning_rate = 0.001), loss=loss, metrics = ['accuracy'])

#Try early stopping to avoid overfitting 
# Define callbacks for avoiding overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

MV_LSTM.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 12, 32)         │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 16)             │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,401 (32.82 KB)

 Trainable params: 8,401 (32.82 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
test5 = MV_LSTM.fit(X_train, y_train, epochs=150, validation_split=0.1, batch_size=32, callbacks=[early_stopping], shuffle=False)

Epoch 1/150
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 0.0236 - root_mean_squared_error: 0.1441 - val_loss: 0.0046 - val_root_mean_squared_error: 0.0675
Epoch 2/150
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 17s 14ms/step - loss: 0.0076 - root_mean_squared_error: 0.0870 - val_loss: 0.0027 - val_root_mean_squared_error: 0.0522
Epoch 3/150
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - loss: 0.0070 - root_mean_squared_error: 0.0837 - val_loss: 0.0017 - val_root_mean_squared_error: 0.0418
Epoch 4/150
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - loss: 0.0068 - root_mean_squared_error: 0.0821 - val_loss: 0.0012 - val_root_mean_squared_error: 0.0339
Epoch 5/150
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - loss: 0.0066 - root_mean_squared_error: 0.0814 - val_loss: 0.0021 - val_root_mean_squared_error: 0.0460
Epoch 6/150
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 17s 14ms/step - loss: 0.0066 - root_mean_squared_error: 0.0809 - val_loss: 0.0011 - val_root_mean_squared_error: 0.0337
Epoch 7/150
1223/1223 

Epoch value max 25

In [ ]:
test_predictions = MV_LSTM.predict(X_test).flatten()
actual_predictions = y_test.flatten()



test_results = pd.DataFrame(data={
        'Model Predictions': test_predictions,
        'Actual':actual_predictions})
test_results

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step


,Model Predictions,Actual
0,0.089810,0.132797
1,0.091292,0.133803
2,0.094786,0.142857
3,0.104015,0.163984
4,0.119621,0.167002
...,...,...
319,-0.004038,0.008048
320,-0.006890,0.010060
321,-0.004901,0.010060
322,-0.004957,0.008048


## Big Failure -Cross Validation
- Only option is to moving window cv when predicting on the test data. In other words, useless.  

In [ ]:

# Walk-forward validation
history = [x for x in X_train]  # Initialize history with the training sequences
predictions = []

for i in range(len(X_test)):
    # Prepare the current input for prediction
    current_input = np.array(history[-window_size])  # Get the last 'window_size' points in history
    current_input = current_input.reshape((1, current_input.shape[0], current_input.shape[1]))
    # Make a prediction
    yhat = MV_LSTM.predict(current_input, verbose=0)
    predictions.append(yhat[0])

    # Append the actual test input to history for the next step (walk-forward)
    history.append(X_test[i])

# Evaluate predictions
predictions = np.array(predictions).flatten()
actual_predictions = y_test.flatten()
predictions.shape, actual_predictions.shape

test_results = pd.DataFrame(data={
        'Model Predictions': predictions,
        'Actual':actual_predictions})


test_rmse = np.sqrt(np.mean((predictions - y_test.flatten()) ** 2))

print(f"Walk-forward validation RMSE: {test_rmse}")

Walk-forward validation RMSE: 0.1112561747431755


# CNN LSTM

In [ ]:
timesteps = X_train.shape[1] #12
n_features = X_train.shape[2] #8
n_seq = 2 #divide 12 time steps by 2 to obtain two subsequences of 6
n_steps= timesteps //n_seq  #6
X_train = X_train.reshape((X_train.shape[0],n_seq, 1 , n_steps , n_features))

In [ ]:
print("X_train shape after reshaping:", X_train.shape)

In [ ]:

MV_LSTM = Sequential()
MV_LSTM.add(Input(shape=(n_seq, 1, n_steps, n_features)))
MV_LSTM.add(ConvLSTM2D(filters=32, kernel_size=(1,2), activation='tanh'))
MV_LSTM.add(Flatten())
MV_LSTM.add(Dense(1))

#Compile the model
metrics="RootMeanSquaredError()"
optimzer="adam"
loss = 'mse'

MV_LSTM.compile(optimizer=Adam(learning_rate = 0.001), loss=loss, metrics = [RootMeanSquaredError()])


MV_LSTM.summary()

In [ ]:
test5 = MV_LSTM.fit(X_train, y_train, epochs=150, validation_split=0.1, batch_size=32, shuffle=False)

In [ ]:
test_predictions = MV_LSTM.predict(X_test).flatten()
actual_predictions = y_test.flatten()



test_results = pd.DataFrame(data={
        'Model Predictions': test_predictions,
        'Actual':actual_predictions})
test_results